In [1]:
pip install pymupdf

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install resume-parser

Processing c:\users\hp\appdata\local\pip\cache\wheels\45\6c\46\a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266\nltk-3.5-py3-none-any.whl
  Created wheel for stemming: filename=stemming-1.0.1-py3-none-any.whl size=11144 sha256=bee96da5825d32abde9d0827c7b0e1df41ef58ef5146a628724ca5f0e69d6d28
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\33\cc\e7\4ad5c9017df40987db1b663d75c3e77667bbaa996475669611
  Created wheel for pdfplumber: filename=pdfplumber-0.5.25-py3-none-any.whl size=31563 sha256=4949c715e4d58eae185b147ec308ff2b0cce17d8b70f55db564f5629dd0180b1
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\8c\65\d1\67ceda23cc7fd669025b485b7f93bea3ee5799a09c95e74452
  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32888 sha256=534d26b3ebfedc4bc4315f5acb5325ed31d78f3e7e5f52368876eff4ed6232b2
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\ec\2b\38\58ff05467a742e32f67f5d0de048fa046e764e2fbb25ac93f3
  Created wheel for smar

ERROR: textract 1.6.3 has requirement pdfminer.six==20181108, but you'll have pdfminer-six 20201018 which is incompatible.
ERROR: tensorflow 2.3.1 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.20.0 which is incompatible.
ERROR: pdfplumber 0.5.25 has requirement pdfminer.six==20200517, but you'll have pdfminer-six 20201018 which is incompatible.
ERROR: typer 0.3.2 has requirement click<7.2.0,>=7.1.1, but you'll have click 7.0 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\hp\\anaconda3\\lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [137]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [138]:
import pandas as pd
import re
import itertools
import warnings
warnings.filterwarnings("ignore")

from resume_parser import resumeparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os,glob


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
mkdir Resume_pdf

In [ ]:
mkdir Resume_txt

**PDF TO TEXT**

In [6]:
import fitz
import os
Txt_dir="/content/Resume_txt"
i=1

for subdir, dirs, files in os.walk('/content/Resume_pdf'):
    for f in files:
      
      PDFfilepath = subdir + os.sep + f
      file=fitz.open(PDFfilepath)
      for pageNumber,page in enumerate(file.pages(),start=1):
            file_name=f.replace(".pdf",".txt");
            TXTfilepath=Txt_dir+os.sep+file_name

            text=page.getText()
            txt=open(TXTfilepath,"a")
            txt.writelines(text)
            txt.close()
    
   
    

mupdf: cannot recognize version marker
mupdf: cannot find startxref


In [141]:
Entity={}
Entity["email"]="([a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+)"
Entity["contact"]='(?i)contact|(?i)phone|[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]'
Entity["linkedin"]="(?!)linkedin\s(?!\w+)|linkedin.com/[^ ]+"
Entity["github"]="(?i)github\s(?!\w+)"
Entity["address"]="(?i)address\s(?!\w+)|(?i)location\s(?!\w+)"
Entity["objective"]="(?i)objective\s(?!\w+)|(?i)SUMMARY\s(?!\w+)|(?i)objective\W|(?i)SUMMARY\W"
Entity["Education"]="(?i)EDUCATION\s(?!\w+)|(?i)EDUCATION:"
Entity["skills"]="(?i)skills\s(?!\w+)|(?i)skills\s(?!\w+)|(?i)skill\sset"
Entity["Tech skills"]="(?i)tech\w+\sskills\s(?!\w+)|(?i)tech\w+\sskills\W"
Entity["Projetcs"]="(?i)projects\s(?!\w+)|(?i)PERSONAL PROJECTS\s(?!\w+)|(?i)Projects\W"
Entity["Internship"]="(?i)internship\s(?!\w+)|(?i)internships\s(?!\w+)"
Entity["Trainings"]="(?i)TRAININGS\s(?!\w+)"
Entity["Work_experience"]="(?i)EXPERIENCE\s(?!\w+)|(?i)Experience:"
Entity["Company"]="(?i)ORGANIZATIONS\s(?!\w+)"
Entity["Awards"]="(?i)achivement\s(?!\w+)|(?i)awards[^\S]|(?i)CERTIFICATES\s(?!\w+)|(?i)CERTIFICATIONS\W"
Entity["Interest"]="(?i)interests\s(?!\w+)|(?i)HOBBIES\s(?!\w+)"
Entity["Language"]="(?i)languages(?!\w+)"
Entity["Work_sample"]="(?i)WORK\sSAMPLES"

**Matching**



> FINAL



In [142]:
file_name=[]
class Resume:
  def __init__(self,filedir,job):
    self.filedir = filedir
    self.job=job

 
  def resume_parser(self):
    final_database=[]
    for filename in glob.glob(os.path.join(self.filedir, '*.txt')):
      with open(filename, 'r') as f:
          file_name.append(filename.replace('/content/Resume_txt/','')) 
        
          lines=f.readlines()
        

          seen = set()
          Heading=[]
          pattern=[]

          #Entities
          for line in lines:
          
          
            for key, value in Entity.items():
              if re.findall(value,str(line)):
                if key not in seen:
                  pattern.append(re.findall(value,line))
                  Heading.append(key)
                  seen.add(key)
          
          #pattern_match
          pattern_list=[]
          for item in pattern:
            pattern_list.append(item[-1].strip())

        

          #insert info at index 0
          Heading.insert(0,"info")
          pattern_list.insert(0,"Info-")

          #split
          f=open(f.name,'r')
          content=f.read().replace("\n"," ")
          
          #Escape character
          match=[]
          for word in pattern_list:
            match.append(re.escape(word))

          split=re.split("|".join(match),content)

          split=list(zip(pattern_list,split))

          for i in range(len(split)):
            split[i]=' '.join(split[i])



          #print frame

          pd.set_option('display.max_colwidth', -1)
          final_database.append(pd.DataFrame(
            {'Heading': Heading ,
            'Content': split}))

          #extract important info
    info=[]
    for i in range(len(final_database)):
      Entities=final_database[i].Heading.values.tolist()
      col=["skills","Tech skills","Projetcs","Work_experience","Language"]#required columns
      resume_skill=[]
      for e in Entities:
        if e in col:
      
          resume_skill.append(str(final_database[i][final_database[i]["Heading"]==e]["Content"].values))
      info.append(''.join(resume_skill))
        
    return info  
        
      
  def job_parser(self):
      data = resumeparse.read_file(self.job)
      job_skill=data["skills"]
      job_skill=' '.join(job_skill)

      return job_skill




  def cleaning_cv(self,docs):
    cleaned=[]

    #for resume
    for doc in docs:
        stop_words = list(set(stopwords.words('english')))
        # split description into words with symbols attached + lower case
        tokenized = word_tokenize(doc.lower())
        #remove stop words
        stopped=[w for w in tokenized if not w in stop_words]
        
        # handle special characters
        cleaned.append(re.sub(r'[^(a-zA-Z)\s]','',str(stopped)))
    return cleaned  

  def cleaning_job(self,doc):
     #for JD   
    cleaned=[]
    stop_words = list(set(stopwords.words('english')))
      # split description into words with symbols attached + lower case
    tokenized = word_tokenize(doc.lower())
    #remove stop words
    stopped=[w for w in tokenized if not w in stop_words]
    # handle special characters
    cleaned.append(re.sub(r'[^(a-zA-Z)\s]','',str(stopped)))

    return cleaned

  

In [143]:
res=Resume("/content/Resume_txt","/content/JD-vDoIT-React_Developer-ggn.pdf")
docs=res.resume_parser()
job=res.job_parser()

In [144]:
cleaned_docs=res.cleaning_cv(docs)
cleaned_jobs=res.cleaning_job(job)

In [145]:
train_set=cleaned_jobs+cleaned_docs

In [146]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize


tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(train_set)]
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")

In [147]:
from gensim.models import doc2vec
from scipy import spatial

d2v_model = doc2vec.Doc2Vec.load("/content/d2v.model")

cos_distance=[]
for i in range(len(cleaned_docs)):
    vec1 = d2v_model.infer_vector(str(cleaned_docs[i]).split())
    vec2 = d2v_model.infer_vector(str(cleaned_jobs).split())

    cos_distance.append(spatial.distance.cosine(vec1, vec2))

In [148]:
resume_match=pd.DataFrame(
            {'Resume': file_name,
            'Score': cos_distance})


In [149]:
resume_match.sort_values(by=['Score'],ascending=False)

,Resume,Score
14,Bijender_Singh_Resume.txt,0.262281
20,Ankit_Gupta_Resume.txt,0.158639
11,Aditya_Kumar_Resume.txt,0.147347
13,Shivam_Bisht_Resume.txt,0.124969
6,Piyush_Sharma_Resume.txt,0.119268
10,Abhishek_Bijalwan_Resume.txt,0.113368
3,Jai_Mehta_Resume.txt,0.109532
8,Naman_Gupta_Resume.txt,0.096847
5,Purvansh_Arora_Resume.txt,0.091825
19,Aarushi_Dua_Resume.txt,0.090221


In [150]:

def resume(mypath):
    filename = mypath.replace("/content/Resume_pdf/","")

    def_matching = resume_match[resume_match["Resume"]==filename.replace('.pdf','.txt')]
    output = {'filename': filename, 'matching%': def_matching}
    return output

In [151]:
resume('/content/Resume_pdf/Aarushi_Dua_Resume.pdf')

{'filename': 'Aarushi_Dua_Resume.pdf',
 'matching%':                     Resume     Score
 19  Aarushi_Dua_Resume.txt  0.090221}